In [1]:
import numpy as np

import chugunov_indicator as chug
from chugunov_indicator.default_data import DefaultScreeningData

import pynucastro as pyna
import yt

In [2]:
# Double Detonation data: http://groot.astro.sunysb.edu/common/
ds = yt.load('./data/subch_plt17526/')
ad = ds.all_data()

yt : [INFO     ] 2024-10-06 20:44:05,053 Parameters: current_time              = 0.800001438968278
yt : [INFO     ] 2024-10-06 20:44:05,053 Parameters: domain_dimensions         = [ 640 1280    1]
yt : [INFO     ] 2024-10-06 20:44:05,055 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-10-06 20:44:05,056 Parameters: domain_right_edge         = [5.12000000e+09 1.02400000e+10 6.28318531e+00]


In [3]:
reaclib_library = pyna.ReacLibLibrary()

nuclei = [field[2:-1] for field in np.array(ds.field_list)[:,1] if "X(" in field]
comp = pyna.Composition(nuclei)

mynet = reaclib_library.linking_nuclei(comp.keys())
pynet = pyna.PythonNetwork(libraries=[mynet])

screen_map = pyna.screening.get_screening_map(
    pynet.get_rates(),
    symmetric_screening=pynet.symmetric_screening
)

In [4]:
comp.set_solar_like()
plasma = pyna.make_plasma_state(1e8, 1e4, comp.get_molar())

In [5]:
pair = screen_map[5]
scn_fac = pyna.make_screen_factors(pair.n1, pair.n2)

In [10]:
%%timeit

F = pyna.screening.chugunov_2009(plasma, scn_fac)

1.85 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
pyna.screening.chugunov_2009(plasma, scn_fac)

1.2989009131276164

In [8]:
screen_interp = DefaultScreeningData.default_interpolator

In [11]:
%%timeit

skip = chug.skip_screening(plasma, scn_fac, screen_interp)

20.6 µs ± 772 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


This means that for individual screening factor calculations, the current interpolator is over 10 times slower than simply computing the screening factor directly.

The bottleneck here is the performance of `intergrid.intergrid.__call__`. While over ten times faster than `scipy.interpolate.RegularGridInterpolator.__call__` , it is still inefficient to use this interpolator to check if the calculation can be skipped.

Possible avenues of improvement include:
- Using an alternative interpolator. While `RegularGridInterpolator` is the best option I could find for Python, it may also be possible to implement something directly in C++ if needed.
- Find numerical models for $c$'s dependence on any of $\bar{A}, \overline{Z^2}, Z_1, Z_2$. This could potentially reduce the dimensionality of the interpolator's inputs or perhaps even circumvent the need for one at all.
- Formulate an analytical model for $c$ by directly manipulating the equations in `chugunov_2009`. This is by far the hardest option given here, but it could be rewarding to try and figure out why the power law $\rho_b \propto T^3$ seems to hold here.